In [1]:
import pandas as pd
import urllib.request
import json
import os
from dotenv import load_dotenv

In [4]:
# .env 파일 로드
load_dotenv()

# 환경변수에서 인증 정보 가져오기
client_id = os.getenv('NAVER_CLIENT_ID')
client_secret = os.getenv('NAVER_CLIENT_SECRET')

# 인증 정보 확인
if not client_id or not client_secret:
    raise ValueError("❌ .env 파일에서 NAVER_CLIENT_ID 또는 NAVER_CLIENT_SECRET를 찾을 수 없습니다!")

print("✅ 환경변수 로드 성공")
print(f"Client ID: {client_id[:5]}... (마스킹)")

# API 요청 URL
url = "https://openapi.naver.com/v1/datalab/search"

# 요청 body (케데헌 검색어 그룹)
body = """{
    "startDate": "2025-01-01",
    "endDate": "2025-12-31",
    "timeUnit": "date",
    "keywordGroups": [
        {
            "groupName": "케이팝데몬헌터스",
            "keywords": ["케이팝 데몬 헌터스", "케이팝데몬헌터스"]
        },
        {
            "groupName": "케데헌",
            "keywords": ["케데헌"]
        },
        {
            "groupName": "국립중앙박물관",
            "keywords": ["국립중앙박물관"]
        }
    ],
    "device": "",
    "ages": [],
    "gender": ""
}"""

print("\n🔄 API 호출 중...")

# API 요청 설정
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id", client_id)
request.add_header("X-Naver-Client-Secret", client_secret)
request.add_header("Content-Type", "application/json")

# API 호출 및 데이터 처리
try:
    response = urllib.request.urlopen(request, data=body.encode("utf-8"))
    rescode = response.getcode()
    
    if rescode == 200:
        response_body = response.read()
        response_data = response_body.decode('utf-8')
        print("✅ API 호출 성공!")
        
        # JSON 파싱
        result = json.loads(response_data)
        
        # 1. 각 키워드 그룹의 데이터를 데이터프레임으로 변환
        df_list = []
        for i in range(len(result['results'])):
            group_name = result['results'][i]['title']
            data = result['results'][i]['data']
            
            if data:
                temp_df = pd.DataFrame(data)
                temp_df.columns = ['date', group_name] # 'period'를 'date'로, 'ratio'를 그룹명으로 변경
                df_list.append(temp_df)
            else:
                print(f"⚠️ '{group_name}' 그룹의 데이터가 없습니다.")

        # 2. 모든 데이터프레임을 'date' 기준으로 합치기
        if df_list:
            from functools import reduce
            df = reduce(lambda left, right: pd.merge(left, right, on='date', how='outer'), df_list)
            
            # 3. 날짜순 정렬 및 데이터가 없는 날은 0으로 채우기
            df = df.sort_values('date').fillna(0)
            
            # CSV 저장 (한글 깨짐 방지)
            filename = 'naver_datalab_케데헌_2025.csv'
            df.to_csv(filename, index=False, encoding='utf-8-sig')

            print("\n📊 데이터 샘플 (처음 10행):")
            print(df.head(10))
            print(f"\n✅ 총 {len(df)}일 데이터 수집 및 병합 완료!")
            print(f"✅ 파일 저장: {filename}")
        else:
            print("❌ 수집된 데이터가 전혀 없습니다.")
            
    else:
        print(f"❌ Error Code: {rescode}")
        
except Exception as e:
    print(f"❌ API 호출 실패: {e}")
    print("💡 팁: .env 파일의 API 키와 네이버 개발자 센터의 [내 애플리케이션 > API 설정]에서 '데이터랩(검색어트렌드)' 권한을 확인하세요.")

✅ 환경변수 로드 성공
Client ID: f9MeR... (마스킹)

🔄 API 호출 중...
✅ API 호출 성공!

📊 데이터 샘플 (처음 10행):
         date  케이팝데몬헌터스  케데헌   국립중앙박물관
0  2025-01-01       0.0  0.0   8.17318
1  2025-01-02       0.0  0.0  11.12598
2  2025-01-03       0.0  0.0  10.24467
3  2025-01-04       0.0  0.0  11.38742
4  2025-01-05       0.0  0.0   9.39077
5  2025-01-06       0.0  0.0  10.11079
6  2025-01-07       0.0  0.0  10.01486
7  2025-01-08       0.0  0.0  10.31531
8  2025-01-09       0.0  0.0   9.24002
9  2025-01-10       0.0  0.0   9.84619

✅ 총 365일 데이터 수집 및 병합 완료!
✅ 파일 저장: naver_datalab_케데헌_2025.csv
